
<body>
    
<h1 style="color:Blue; font-family: Roboto, sans-serif; text-align:center; font-size:30px">Practice your ML regression skills on this approachable dataset! 📚💬</h1>
    
</body>


<center>
    <img src= "https://storage.googleapis.com/kaggle-competitions/kaggle/24673/logos/header.png" width="600">
</center>
<center>
    <img src= "https://storage.googleapis.com/molten/lava/2019/05/48c835e7-machine-learning-gif.gif" width="600">
</center>

<h1 style="color:RED; text-align:left; font-family: 'Oswald', sans-serif "> ABOUT: </h1>

<h3 style="text-align:Left">Hi everyone, I want to share with you all my data exploration, analysis and model selection about this topic, I've been scraching this dataset for a while, and result very interesting for start into data science field. I'm really looking forwards about your comments and recomendation for this project.
</h3>


<h2 style=" color:Blue; text-align:center; font-family:'Lobster',">Let's do it!</h2>

# Table of Contents
1. [Chapter 1 - What is the problem?](#ch1)
1. [Chapter 2 - Data Science Pipeline](#ch2)
1. [Chapter 3 - Overview data](#ch3)
1. [Chapter 4 - Prepare data for consumption](#ch4)
1. [Chapter 5 - Exploratory Data Analysis](#ch5)
1. [Chapter 6 - Model Data](#ch6)
1. [Chapter 7 - What did we find about diabetes?](#ch7)
1. [Submission File](#ch99)

<a id="ch1"></a>
# CHAPTER 1. What is the problem?

Tabular Playground Series - Jan 2021, is an entry project to show our data science skills to progress and learn about the field. The project demands us a regression approach to predict the label data, and yes, this is another supervised learning competition.

<a id="ch2"></a>
# CHAPTER 2. Data Science Pipeline

1. **Define the Problem:** We face a blind columns data, for a regression problem.

2. **Gather the Data:** The challenge provided us with structured data, hence we can accomplish the problem like a [supervised learning](https://machinelearningmastery.com/supervised-and-unsupervised-machine-learning-algorithms/).

3. **Prepare Data for Consumption:** This step could overlap exploratory data analysis EDA, while our exploration we'll need to clean, transform, reformat or scale our dataset, all this for extract useful information to clarify our problem and have healthy data, to avoid missing or outlier data points.

4. **Perform Exploratory Analysis:** Without analysis our work could be useless, because "garbage-in, garbage-out (GIGO)". This is the step when we going to create our personal perspective of the problem but trying really hard to be objective, sometimes we have to use statistics tools to validate some hypothesis. All this analysis will be support by graphics aims to looking patterns, comparations and correlations. Also, we going to need to differentiate categorical vs numerical features, and balance of those. These aspects of the data are important to create the best hypothesis for our data model.

5. **Model Data:** The data model that we'll chose determine the algorithms available for use. The algorithm is just a fancy tool completely useless in the wrong context. That’s because data modeling came after EDA, first you understand the data, then you model it.

6. **Validate and Implement Data Model:** After you've trained your model based on a subset of your data, it's time to test your model. This helps ensure you haven't overfit your model or made it so specific to the selected subset, that it does not accurately fit another subset from the same dataset. In this step we determine if our [model overfit, generalize, or underfit our dataset](https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/).

7. **Optimize and Strategize:** After all these steps, we'll need to repeat it, creating different hypothesis and models, optimize our process aims to make better predictions.

<a id="ch3"></a>
# CHAPTER 3. Overview Data

For this competition, you will be predicting a continuous target based on a number of feature columns given in the data. All of the feature columns, cont1 - cont14 are continuous.

## Files
* **train.csv** - the training data with the target column
* **test.csv** - the test set; you will be predicting the target for each row in this file
* **sample_submission.csv** - a sample submission file in the correct format

In [ ]:
import numpy as np 
import pandas as pd 

# Input data files are available in the read-only "../input/" directory
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# import libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

import IPython.display as display

# Graphics libraries
#!pip install pywaffle
#!pip install pycomp

import shap

#from pywaffle import Waffle
import xgboost
import pickle

import warnings
#warnings.filterwarnings("ignore")

In [ ]:
def missing_count(df):
    missing_count = df.isna().sum()
    missing_df = (pd.concat([missing_count.rename('Missing count'),
                     missing_count.div(len(df))
                          .rename('Missing ratio')],axis = 1)
             .loc[missing_count.ne(0)])
    return missing_df.sort_values(by="Missing ratio")

### Paths 📚

In [ ]:
SampleSubmission = "../input/tabular-playground-series-jan-2021/sample_submission.csv"
traintablepath = "../input/tabular-playground-series-jan-2021/train.csv"
testtablepath = "../input/tabular-playground-series-jan-2021/test.csv"

### Open files

In [ ]:
table_train = pd.read_csv(traintablepath)
table_test =  pd.read_csv(testtablepath)
sampSum = pd.read_csv(SampleSubmission)

## Train file overview

1. Description table shows us a simple distribution behavior for every feature of the data.
2. Missing rate values present on our dataset.
3. Sample train table, only five rows.


In [ ]:
missing_df = missing_count(table_train)

display.display(table_train.describe().T)
display.display(missing_df)
table_train.head()

## Test file overview

In [ ]:
missing_df = missing_count(table_test)

display.display(table_test.describe().T)
display.display(missing_df)
table_test.head()

<a id="ch4"></a>
# CHAPTER 4. Prepare data for consumption
1. If we review the chapter before, we can't find any nan value, is a very idealistic situation, but is good for us.
1. There is a feature which is useless, "id", so we need to drop it. The unique identifiers are just noise.
1. We need to see what type of data there in our dataset

### Droping useless column


In [ ]:
columns = ["id"]
table_train = table_train.drop(columns=columns)
table_test = table_test.drop(columns=columns)

### Differentiate between Numerical and Categorical Data.

We see that that this data set only contain real float64 columns. So let's work with that.


In [ ]:
table_train.dtypes

<a id="ch5"></a>
# CHAPTER 5. Exploratory Data Analysis
1. We can see that our data need to be normalize
1. There are too many irregular behaviors that we need to correct on the preprocessing steps


In [ ]:
def plot_whole_features(df):
    columns_num = df.columns
    fig, axes = plt.subplots(nrows=round(len(columns_num)/3), ncols=3, figsize=(40, 40))
    for ax, column in zip(axes.flatten(),columns_num):
        sns.histplot(df[column], ax=ax )
        sns.set(font_scale=1)

In [ ]:
# Create distplot with curve_type set to 'normal'
def hist_plotly(df, col,bin_size=5,color = "slategray"):
    colors = [color]
    fig = ff.create_distplot([df[col].dropna()], [col],bin_size=bin_size,
                             curve_type='kde', # override default 'kde'
                             show_rug=False,
                             colors=colors)

    # Add title
    fig.update_layout(title_text='{} - Distplot with Normal Distribution'.format(col.upper()) ,
                  template= "plotly_dark" , 
                  xaxis = dict(title = col.upper()))
    fig.show()

## Output Target Distribution
1. we see that this output is very tricky, the behavior is not normalized and so many outliers.
1. A bimodal behavior could be a problem for a good performance.

In [ ]:
hist_plotly(table_train, "target", bin_size = .01)

## Features

In [ ]:
plot_whole_features(table_train)

## Exploring Correlation Matrix

For a fast clean visualization about the relation between columns we can use our best friend, the correlation matrix.


In [ ]:
table_train.corr(method = "pearson").style.background_gradient(cmap='Reds')

In [ ]:
# Compute the correlation matrix
corr = table_train.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 12))
ax.title.set_text('Corr Matrix - Whole Features')

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
_ = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.,vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})


In [ ]:
corr_featu=["cont1", "cont2", "cont3", "cont4", "cont14" , "cont8", "cont9", "cont12" ]

<a id="ch6"></a>
# CHAPTER 6. Model Data

In [ ]:
from sklearn.preprocessing import PowerTransformer, PolynomialFeatures
from sklearn.preprocessing import QuantileTransformer

def poly_augm(df, columns, encoder, suff= "_poly"):
    df = df.join(pd.DataFrame(encoder.transform(df[columns]), columns = encoder.get_feature_names(columns) ).add_suffix(suff))
    return df


## Delete the outlier value in the dataset

In [ ]:
table_train = table_train[table_train["target"]>1]

### Feature Generation

Of course, i've created  more features, i'd considered the most inner interactive features.

["cont1", "cont6", "cont7", "cont8", "cont9", "cont10", "cont11", "cont12" ]


In [ ]:
poly = PolynomialFeatures(3,interaction_only = True)
poly.fit(table_train[corr_featu])

table_train_agg = poly_augm(table_train, corr_featu, poly)
table_test_agg = poly_augm(table_test, corr_featu, poly)

table_train_agg.head()

### Scaling Data
1. QuantileTransformer

Transform features using quantiles information.

This method transforms the features to follow a uniform or a normal distribution. Therefore, for a given feature, this transformation tends to spread out the most frequent values. It also reduces the impact of (marginal) outliers: this is therefore a robust preprocessing scheme.

The transformation is applied on each feature independently. First an estimate of the cumulative distribution function of a feature is used to map the original values to a uniform distribution. The obtained values are then mapped to the desired output distribution using the associated quantile function. Features values of new/unseen data that fall below or above the fitted range will be mapped to the bounds of the output distribution. Note that this transform is non-linear. It may distort linear correlations between variables measured at the same scale but renders variables measured at different scales more directly comparable.

In [ ]:
# Train
train_model_sc = table_train_agg.copy()
train_label = train_model_sc["target"]
train_model_sc = train_model_sc.drop(columns = ["target"])
columns = train_model_sc.columns
# Test
test_model_sc = table_test_agg.copy()
#Scaler
qt = QuantileTransformer(n_quantiles=1000, random_state=0,output_distribution = "normal")
# Scaling
train_model_sc[columns] = qt.fit_transform(train_model_sc)
test_model_sc[columns]  = qt.transform(test_model_sc)

train_model_sc.head()

## Model Time

### Training - Model Optimization

This step will help us to chose the best hyperparams for our model.
Be awere that this process is time computing expensive. so, take a rest and do anything else.

#### **RandomizedSearchCV**

Randomized search on hyper parameters.

RandomizedSearchCV implements a “fit” and a “score” method. It also implements “score_samples”, “predict”, “predict_proba”, “decision_function”, “transform” and “inverse_transform” if they are implemented in the estimator used.

The parameters of the estimator used to apply these methods are optimized by cross-validated search over parameter settings.

In contrast to GridSearchCV, not all parameter values are tried out, but rather a fixed number of parameter settings is sampled from the specified distributions. The number of parameter settings that are tried is given by n_iter.

Let's look [xgboost classifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn)

In [ ]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV , RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.metrics import r2_score , explained_variance_score , mean_absolute_error , mean_squared_error
from IPython.display import display
from lightgbm import LGBMRegressor

def regression_error(y_true, y_pred):
    r2 = r2_score(y_true, y_pred )
    evs = explained_variance_score(y_true, y_pred )
    mae = mean_absolute_error(y_true, y_pred )
    mse = mean_squared_error(y_true, y_pred )
    data = [[r2, ""],[evs,""] ,[ "", mae],[ "", mse]]
    info = pd.DataFrame(data =data, index = ['R2','ExplaVarScore', 'MeanAbsErr', 'MeanSquaredErr'], columns =['Score', 'Error'])
    display(info)
    

X_train, X_test, y_train, y_test = train_test_split(train_model_sc, train_label, train_size=0.8)

### XGB model Randomized Search

In [ ]:
tuned_parameters_xgb = [{'tree_method': ['gpu_hist'],'booster':['gbtree'], 'predictor':['gpu_predictor'], 'learning_rate':np.linspace(.05, 1, 20),'min_child_weight':np.linspace(0, 5, 50), 'n_estimators':list(range(50, 300, 20)), # 'subsample':[0.1, 0.5, 0.8,], 'n_estimators':[100,150], 'min_child_weight':[0.1, 0.5, 1.0,],
                    'max_depth': list(range(3,15)), 'gamma': np.linspace(0,1.,20) }]

scores = {'r2': 'r2', 'nmae':'neg_median_absolute_error' , 'nrmse': 'neg_root_mean_squared_error'}     
# GridSearchCV
clf = RandomizedSearchCV(XGBRegressor(), tuned_parameters_xgb,
                   scoring=scores, 
                   refit= "nrmse",
                   cv=7,
                   verbose=2,
                   n_jobs=2,
                   n_iter= 40,)

clf.fit(X_train, y_train)
# Validation
pred = clf.best_estimator_.predict(X_test)
print(regression_error(y_test, pred ))
print(clf.best_params_)

### LGBM model Randomized Search

In [ ]:
tuned_parameters_lgb = [{'metric':['rmse'],'objective': ['regression'], 'learning_rate':np.linspace(.01, 1, 20),'min_gain_to_split':np.linspace(0, 5, 50), 'num_leaves':list(range(20, 200, 20)), # 'subsample':[0.1, 0.5, 0.8,], 'n_estimators':[100,150], 'min_child_weight':[0.1, 0.5, 1.0,],
                    'max_depth': list(range(3,15)), 'max_bin': list(range(50,1000, 50)) }]

scores_lgb = {'r2': 'r2', 'nmae':'neg_median_absolute_error' , 'nrmse': 'neg_root_mean_squared_error'}     
# GridSearchCV
clf_lgb = RandomizedSearchCV(LGBMRegressor(), tuned_parameters_lgb,
                   scoring=scores, 
                   refit= "nrmse",
                   cv=7,
                   verbose=2,
                   n_jobs=2,
                   n_iter= 40,)

clf_lgb.fit(X_train, y_train)
# Validation
pred = clf_lgb.best_estimator_.predict(X_test)
print(regression_error(y_test, pred ))
print(clf_lgb.best_params_)

In [ ]:
clf_lgb.cv_results_

In [ ]:
params_lgb = clf_lgb.best_params_ 

In [ ]:
clf.cv_results_

In [ ]:
params= clf.best_params_ 

## Final Training
After run and succesfuly finished the RandomSearch, we need to make the last training with the whole data set, using the best hyperparameters.

In [ ]:
model_lgb = LGBMRegressor(**params_lgb)
model_lgb.fit(train_model_sc, train_label)

In [ ]:
model = XGBRegressor(**params)
model.fit(train_model_sc, train_label)

<a id="ch99"></a>

## Submission File

Well folks, this is it. Is this was useful for you, plesase give an upvote and let me a comment about how to improve this notebook. Thanks

In [ ]:
# LGB
y_pred     = model_lgb.predict(test_model_sc)

encounter_IDs = pd.read_csv(testtablepath)[["id"]].values
df_sub = {'id': encounter_IDs[:,0], 'target': y_pred}
df_predictions = pd.DataFrame.from_dict(df_sub).set_index(['id'])
df_predictions.to_csv('Predictions_final_lgb.csv')

df_predictions.head(10)

In [ ]:
# XGB
y_pred     = model.predict(test_model_sc)

encounter_IDs = pd.read_csv(testtablepath)[["id"]].values
df_sub = {'id': encounter_IDs[:,0], 'target': y_pred}
df_predictions = pd.DataFrame.from_dict(df_sub).set_index(['id'])
df_predictions.to_csv('Predictions_final.csv')

df_predictions.head(10)

## Stacking Regressor
Now i going to stack bouth regressors using staking regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor   as DTR
from sklearn.ensemble import StackingRegressor as SR
model_sc = SR(estimators = [('xgb' ,model), ('lgb' ,model_lgb)],final_estimator= DTR(random_state=17,max_depth=4,criterion= "mse"),cv = 8 )
model_sc.fit(X_train, y_train)
pred = model_sc.predict(X_test)
print(regression_error(y_test, pred ))

In [ ]:
model_sc.fit(train_model_sc, train_label)

In [ ]:
# Staking Regressor
y_pred     = model_sc.predict(test_model_sc)

encounter_IDs = pd.read_csv(testtablepath)[["id"]].values
df_sub = {'id': encounter_IDs[:,0], 'target': y_pred}
df_predictions = pd.DataFrame.from_dict(df_sub).set_index(['id'])
df_predictions.to_csv('Predictions_final_sc.csv')

df_predictions.head(10)

<a id="ch8"></a>

# Chapter 7 - What did we find about diabetes

## Feature importance

Let's see what can tell us our model. What features are relevant for the task.

This step will be implemented using SHAP, who use game theory for extract information about our model, we could’ve used the feature importance calculations that come with XGBoost, but it’s not accurate at all or even contradictory.

If you want to see more about SHAP, please [check the link. ](https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/tree_based_models/Census%20income%20classification%20with%20XGBoost.html)

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)
shap.summary_plot(shap_values, X_test, plot_type="bar",
                  title = "")

In [ ]:
shap.summary_plot(shap_values, X_test)